###                                  <center> CSE555 – Deep Learning (Spring 2018) </center>
###                                               <center> Homework 3 </center>
###                                    <center>   Harlinton Palacios Mosquera </center>
                                   
###                                       <center>  ID: 161041033  </center>  

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Activation
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Conv1D ,GlobalMaxPooling1D ,Dropout
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np

C:\Users\Harlinton\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words    
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

This function generate a sequence from a language model

In [3]:
import collections, nltk


def unigram(tokens):    
    model0 = collections.defaultdict(lambda: 0.01)
    for f in tokens:
        #print(f)
        try:
            model0[f] += 1
        except KeyError:
            model0 [f] = 1
            continue
    for word in model0:
        model0[word] = model0[word]/float(sum(model0.values()))
    return model0

Here you construct the unigram language model 

In [4]:

def perplexity(testset, model0):
    testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model0[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

Computes perplexity of the unigram model on a testset  

#  Part 1. Prepare data

In [30]:
with open('./data/trwiki') as f:
    data = f.readlines()    
print (data)
sentences = []
import re
# Go through each text in turn
for i in range(len(data)):
    sentences = [re.sub(pattern=r'\!"#$%&;\*+,-./:;&lt;=&;?@^_`()|~=', 
                        repl='', 
                        string=x
                       ).strip().split(' ') for x in data[i].split('\n') 
                      if not x.endswith('writes:')]
    sentences = [x for x in sentences if x != ['']]
    data[i] = sentences

corpus1 = data[0]


['Temmuz ayinda Marx ve Bauer Berlin den Bonn a bir yolculuk yaptilar.Orada sarhos olmaktan, kilisede kahkaha ile gülmeye ve sehirde esek turu atmaya kadar skandal sayilabilecek herseyi yaptilar.Marx akademik bir kariyer planlamasina ragmen hükümetin klasik liberalizme ve Genç Hegelciler e karsi artan tepkisi nedeniyle bu seçenegin önü tikanmisti. Sosyalizm hakkindaki ilk fikirlerini ve ekonomiye artan ilgisini yazacagi radikal bir gazete olan\n', 'Rheinische Zeitung "Rhineland News" gazetecilik yapmak üzere Marx de Köln e gitti. Hem sag kanat Avrupa hükümetlerini hem de liberal ve sosyalist hareketler içindeki çesitli kisileri etkisiz ve üretkenlik karsiti olmalari nedeniyle elestirdi ']


In this section load the data and cleaned the data if it has characters not necessary for data processing and seleced a paragraf of the data to training a NNs

In [16]:
# source text
corpus = """ Temmuz ayinda Marx ve Bauer Berlin den Bonn a bir yolculuk yaptilar.Orada sarhos olmaktan, kilisede kahkaha ile gülmeye ve sehirde esek turu atmaya kadar skandal sayilabilecek herseyi yaptilar.
Marx akademik bir kariyer planlamasina ragmen hükümetin klasik liberalizme ve Genç Hegelciler e karsi artan tepkisi nedeniyle 
bu seçenegin önü tikanmisti. Sosyalizm hakkindaki ilk fikirlerini ve ekonomiye artan ilgisini yazacagi radikal bir gazete olan
Rheinische Zeitung "Rhineland News" gazetecilik yapmak üzere Marx de Köln e gitti. Hem sag kanat Avrupa hükümetlerini 
hem de liberal ve sosyalist hareketler içindeki çesitli kisileri etkisiz ve üretkenlik karsiti olmalari nedeniyle elestirdi.
"""
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
encoded = tokenizer.texts_to_sequences([corpus])[0]
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
    sequence = encoded[i-2:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = (sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)


Vocabulary Size: 82
Total Sequences: 94
Max Sequence Length: 3


<b>Coment:</b> In this section we using pre-trained word vectors in the embedding. Keras provides the <b>Tokenizer</b> class that can be used to perform this encoding. First, the Tokenizer is fit on the source text to develop the mapping from words to unique integers. Then sequences of text can be converted to sequences of integers by calling the texts_to_sequences() function.

In [17]:
embedding_layer = Embedding(vocab_size, 3,input_length=max_length-1)

# Part 2.1 Next Word Prediction using CNN

In [18]:
model_1 = Sequential()
model_1.add(embedding_layer)
model_1.add(Conv1D(250,2,padding='valid',activation='relu',strides=1))
model_1.add(GlobalMaxPooling1D())
model_1.add(Dense(250))
model_1.add(Dropout(0.2))
model_1.add(Activation('relu'))
model_1.add(Dense(vocab_size, activation ='sigmoid'))
model_1.compile(loss='categorical_crossentropy',optimizer='rmsprop')
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 3)              246       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 250)            1750      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 82)                20582     
Total para

In [19]:
model_1.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 0s - loss: 4.4083
Epoch 2/500
 - 0s - loss: 4.4036
Epoch 3/500
 - 0s - loss: 4.4005
Epoch 4/500
 - 0s - loss: 4.3952
Epoch 5/500
 - 0s - loss: 4.3897
Epoch 6/500
 - 0s - loss: 4.3829
Epoch 7/500
 - 0s - loss: 4.3748
Epoch 8/500
 - 0s - loss: 4.3645
Epoch 9/500
 - 0s - loss: 4.3502
Epoch 10/500
 - 0s - loss: 4.3364
Epoch 11/500
 - 0s - loss: 4.3191
Epoch 12/500
 - 0s - loss: 4.2977
Epoch 13/500
 - 0s - loss: 4.2792
Epoch 14/500
 - 0s - loss: 4.2570
Epoch 15/500
 - 0s - loss: 4.2273
Epoch 16/500
 - 0s - loss: 4.2050
Epoch 17/500
 - 0s - loss: 4.1798
Epoch 18/500
 - 0s - loss: 4.1628
Epoch 19/500
 - 0s - loss: 4.1298
Epoch 20/500
 - 0s - loss: 4.0869
Epoch 21/500
 - 0s - loss: 4.0515
Epoch 22/500
 - 0s - loss: 4.0041
Epoch 23/500
 - 0s - loss: 3.9843
Epoch 24/500
 - 0s - loss: 3.9428
Epoch 25/500
 - 0s - loss: 3.8980
Epoch 26/500
 - 0s - loss: 3.8361
Epoch 27/500
 - 0s - loss: 3.7855
Epoch 28/500
 - 0s - loss: 3.7324
Epoch 29/500
 - 0s - loss: 3.6782
Epoch 30/500
 - 0s - lo

 - 0s - loss: 0.0175
Epoch 239/500
 - 0s - loss: 0.0252
Epoch 240/500
 - 0s - loss: 0.0088
Epoch 241/500
 - 0s - loss: 0.0179
Epoch 242/500
 - 0s - loss: 0.0128
Epoch 243/500
 - 0s - loss: 0.0481
Epoch 244/500
 - 0s - loss: 0.0179
Epoch 245/500
 - 0s - loss: 0.0105
Epoch 246/500
 - 0s - loss: 0.0092
Epoch 247/500
 - 0s - loss: 0.0094
Epoch 248/500
 - 0s - loss: 0.0228
Epoch 249/500
 - 0s - loss: 0.0262
Epoch 250/500
 - 0s - loss: 0.0146
Epoch 251/500
 - 0s - loss: 0.0118
Epoch 252/500
 - 0s - loss: 0.0301
Epoch 253/500
 - 0s - loss: 0.0109
Epoch 254/500
 - 0s - loss: 0.0092
Epoch 255/500
 - 0s - loss: 0.0097
Epoch 256/500
 - 0s - loss: 0.0124
Epoch 257/500
 - 0s - loss: 0.0090
Epoch 258/500
 - 0s - loss: 0.0094
Epoch 259/500
 - 0s - loss: 0.0351
Epoch 260/500
 - 0s - loss: 0.0154
Epoch 261/500
 - 0s - loss: 0.0069
Epoch 262/500
 - 0s - loss: 0.0075
Epoch 263/500
 - 0s - loss: 0.0158
Epoch 264/500
 - 0s - loss: 0.0175
Epoch 265/500
 - 0s - loss: 0.0043
Epoch 266/500
 - 0s - loss: 0.0095

Epoch 462/500
 - 0s - loss: 2.3570e-04
Epoch 463/500
 - 0s - loss: 0.0015
Epoch 464/500
 - 0s - loss: 1.4003e-04
Epoch 465/500
 - 0s - loss: 5.6349e-04
Epoch 466/500
 - 0s - loss: 0.0017
Epoch 467/500
 - 0s - loss: 3.4892e-04
Epoch 468/500
 - 0s - loss: 1.4936e-04
Epoch 469/500
 - 0s - loss: 6.0980e-05
Epoch 470/500
 - 0s - loss: 9.1461e-05
Epoch 471/500
 - 0s - loss: 1.7969e-04
Epoch 472/500
 - 0s - loss: 1.8207e-04
Epoch 473/500
 - 0s - loss: 5.4592e-05
Epoch 474/500
 - 0s - loss: 1.2117e-04
Epoch 475/500
 - 0s - loss: 4.1530e-04
Epoch 476/500
 - 0s - loss: 0.0012
Epoch 477/500
 - 0s - loss: 1.7051e-04
Epoch 478/500
 - 0s - loss: 1.5703e-04
Epoch 479/500
 - 0s - loss: 8.9952e-05
Epoch 480/500
 - 0s - loss: 5.6559e-05
Epoch 481/500
 - 0s - loss: 2.0393e-04
Epoch 482/500
 - 0s - loss: 1.8721e-04
Epoch 483/500
 - 0s - loss: 1.0895e-04
Epoch 484/500
 - 0s - loss: 1.7441e-04
Epoch 485/500
 - 0s - loss: 2.1135e-04
Epoch 486/500
 - 0s - loss: 4.7404e-05
Epoch 487/500
 - 0s - loss: 1.5352e-0

In [76]:
predicNCC= (generate_seq(model_1, tokenizer, max_length-1, "Temmuz ayinda Marx ve", 2))
print('Next Word Prediction : %s' % predicNCC)

Next Word Prediction : Temmuz ayinda Marx ve bauer berlin


<b>Coment :</b>In this section we use the <b>CNN model</b> to predict the next word. In addition, evaluate the model using the Generate_Seq function. which generates the prediction of the next word. In this case I used the previous 4 words and predicted the next word  ()

<b>Previous 4 word </b>: Temmuz ayinda Marx ve

<b>2 Predict word  </b>: bauer berlin

# Part 2.2 Next Word Prediction using  LSTM

In [21]:
# define model
model = Sequential()

# Add Input Embedding Layer
model.add(Embedding(vocab_size, 10, input_length=max_length-1))

# Add Hidden Layer 1 - LSTM Layer
model.add(LSTM(100))

# Add Output Layer
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2, 10)             820       
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_5 (Dense)              (None, 82)                8282      
Total params: 53,502
Trainable params: 53,502
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 4.4073 - acc: 0.0213
Epoch 2/500
 - 0s - loss: 4.4054 - acc: 0.0319
Epoch 3/500
 - 0s - loss: 4.4039 - acc: 0.0532
Epoch 4/500
 - 0s - loss: 4.4022 - acc: 0.0638
Epoch 5/500
 - 0s - loss: 4.4006 - acc: 0.0638
Epoch 6/500
 - 0s - loss: 4.3988 - acc: 0.0638
Epoch 7/500
 - 0s - loss: 4.3971 - acc: 0.0638
Epoch 8/500
 - 0s - loss: 4.3952 - acc: 0.0638
Epoch 9/500
 - 0s - loss: 4.3931 - acc: 0.0638
Epoch 10/500
 - 0s - loss: 4.3904 - acc: 0.0638
Epoch 11/500
 - 0s - loss: 4.3880 - acc: 0.0638
Epoch 12/500
 - 0s - loss: 4.3852 - acc: 0.0638
Epoch 13/500
 - 0s - loss: 4.3819 - acc: 0.0638
Epoch 14/500
 - 0s - loss: 4.3783 - acc: 0.0638
Epoch 15/500
 - 0s - loss: 4.3742 - acc: 0.0638
Epoch 16/500
 - 0s - loss: 4.3693 - acc: 0.0638
Epoch 17/500
 - 0s - loss: 4.3637 - acc: 0.0638
Epoch 18/500
 - 0s - loss: 4.3579 - acc: 0.0638
Epoch 19/500
 - 0s - loss: 4.3513 - acc: 0.0638
Epoch 20/500
 - 0s - loss: 4.3438 - acc: 0.0638
Epoch 21/500
 - 0s - loss: 4.3345 - acc: 0.0638
E

Epoch 171/500
 - 0s - loss: 0.1016 - acc: 1.0000
Epoch 172/500
 - 0s - loss: 0.0999 - acc: 1.0000
Epoch 173/500
 - 0s - loss: 0.0982 - acc: 1.0000
Epoch 174/500
 - 0s - loss: 0.0963 - acc: 1.0000
Epoch 175/500
 - 0s - loss: 0.0948 - acc: 1.0000
Epoch 176/500
 - 0s - loss: 0.0930 - acc: 1.0000
Epoch 177/500
 - 0s - loss: 0.0915 - acc: 1.0000
Epoch 178/500
 - 0s - loss: 0.0900 - acc: 1.0000
Epoch 179/500
 - 0s - loss: 0.0884 - acc: 1.0000
Epoch 180/500
 - 0s - loss: 0.0871 - acc: 1.0000
Epoch 181/500
 - 0s - loss: 0.0855 - acc: 1.0000
Epoch 182/500
 - 0s - loss: 0.0842 - acc: 1.0000
Epoch 183/500
 - 0s - loss: 0.0829 - acc: 1.0000
Epoch 184/500
 - 0s - loss: 0.0816 - acc: 1.0000
Epoch 185/500
 - 0s - loss: 0.0802 - acc: 1.0000
Epoch 186/500
 - 0s - loss: 0.0790 - acc: 1.0000
Epoch 187/500
 - 0s - loss: 0.0777 - acc: 1.0000
Epoch 188/500
 - 0s - loss: 0.0765 - acc: 1.0000
Epoch 189/500
 - 0s - loss: 0.0753 - acc: 1.0000
Epoch 190/500
 - 0s - loss: 0.0741 - acc: 1.0000
Epoch 191/500
 - 0s 

 - 0s - loss: 0.0166 - acc: 1.0000
Epoch 339/500
 - 0s - loss: 0.0165 - acc: 1.0000
Epoch 340/500
 - 0s - loss: 0.0164 - acc: 1.0000
Epoch 341/500
 - 0s - loss: 0.0163 - acc: 1.0000
Epoch 342/500
 - 0s - loss: 0.0162 - acc: 1.0000
Epoch 343/500
 - 0s - loss: 0.0161 - acc: 1.0000
Epoch 344/500
 - 0s - loss: 0.0159 - acc: 1.0000
Epoch 345/500
 - 0s - loss: 0.0158 - acc: 1.0000
Epoch 346/500
 - 0s - loss: 0.0157 - acc: 1.0000
Epoch 347/500
 - 0s - loss: 0.0156 - acc: 1.0000
Epoch 348/500
 - 0s - loss: 0.0155 - acc: 1.0000
Epoch 349/500
 - 0s - loss: 0.0154 - acc: 1.0000
Epoch 350/500
 - 0s - loss: 0.0153 - acc: 1.0000
Epoch 351/500
 - 0s - loss: 0.0152 - acc: 1.0000
Epoch 352/500
 - 0s - loss: 0.0151 - acc: 1.0000
Epoch 353/500
 - 0s - loss: 0.0150 - acc: 1.0000
Epoch 354/500
 - 0s - loss: 0.0149 - acc: 1.0000
Epoch 355/500
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 356/500
 - 0s - loss: 0.0147 - acc: 1.0000
Epoch 357/500
 - 0s - loss: 0.0146 - acc: 1.0000
Epoch 358/500
 - 0s - loss: 0.0145

In [75]:
predictLSTM = (generate_seq(model, tokenizer, max_length-1, "Berlin den Bonn a bir ", 1))
print('Next Word Prediction : %s' % predictLSTM)

Next Word Prediction : Berlin den Bonn a bir  yolculuk


<b>Coment :</b>In this section we use the <b>LSTM model</b> to predict the next word. In addition, evaluate the model using the Generate_Seq function. which generates the prediction of the next word. In this case I used the previous 4 words and predicted the next word 

<b>Previous 4 word </b>: Berlin den Bonn a bir

<b>Predict word  </b>: yolculuk

# Part 3.  Next Word Prediction Given Partial Word

In [45]:
hola = "planlamasina ragmen"
xlstm = list(hola)
# integer encode sequences of words
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts([xlstm])
encoded1 = tokenizer1.texts_to_sequences([xlstm])[0]

# retrieve vocabulary size
vocab_size1 = len(tokenizer1.word_index) + 1
print('Vocabulary Size: %d' % vocab_size1)
# encode 2 words -> 1 word
sequences1 = list()
for i in range(2, len(encoded1)):
	sequence1 = encoded1[i-2:i+1]
	sequences1.append(sequence1)
print('Total Sequences: %d' % len(sequences1))
# pad sequences
max_length1 = max([len(seq) for seq in sequences1])
sequences1 = pad_sequences(sequences1, maxlen=max_length1, padding='pre')
print('Max Sequence Length: %d' % max_length1)
# split into input and output elements
sequences1 = (sequences1)
Xx, yy = sequences1[:,:-1],sequences1[:,-1]
Xx, yy = sequences1[:,0],sequences1[:,1]
yy = to_categorical(yy, num_classes=vocab_size1)
print(Xx[:5])
yy[:5]

Vocabulary Size: 12
Total Sequences: 17
Max Sequence Length: 3
[5 3 1 2 3]


array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# LSTM 

In [46]:
# define model
model2 = Sequential()

# Add Input Embedding Layer
model2.add(Embedding(vocab_size1, 10, input_length=1))

# Add Hidden Layer 1 - LSTM Layer
model2.add(LSTM(256))

# Add Output Layer
model2.add(Dense(vocab_size1, activation='softmax'))

print(model2.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1, 10)             120       
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dense_8 (Dense)              (None, 12)                3084      
Total params: 276,612
Trainable params: 276,612
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
# compile network
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model2.fit(Xx, yy, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 2.4843 - acc: 0.1765
Epoch 2/500
 - 0s - loss: 2.4826 - acc: 0.2353
Epoch 3/500
 - 0s - loss: 2.4807 - acc: 0.2941
Epoch 4/500
 - 0s - loss: 2.4786 - acc: 0.2941
Epoch 5/500
 - 0s - loss: 2.4764 - acc: 0.2941
Epoch 6/500
 - 0s - loss: 2.4741 - acc: 0.2941
Epoch 7/500
 - 0s - loss: 2.4717 - acc: 0.2941
Epoch 8/500
 - 0s - loss: 2.4692 - acc: 0.2941
Epoch 9/500
 - 0s - loss: 2.4666 - acc: 0.2941
Epoch 10/500
 - 0s - loss: 2.4638 - acc: 0.2941
Epoch 11/500
 - 0s - loss: 2.4610 - acc: 0.2941
Epoch 12/500
 - 0s - loss: 2.4581 - acc: 0.2941
Epoch 13/500
 - 0s - loss: 2.4550 - acc: 0.2941
Epoch 14/500
 - 0s - loss: 2.4518 - acc: 0.2941
Epoch 15/500
 - 0s - loss: 2.4484 - acc: 0.2941
Epoch 16/500
 - 0s - loss: 2.4449 - acc: 0.2941
Epoch 17/500
 - 0s - loss: 2.4413 - acc: 0.2941
Epoch 18/500
 - 0s - loss: 2.4375 - acc: 0.2941
Epoch 19/500
 - 0s - loss: 2.4335 - acc: 0.2941
Epoch 20/500
 - 0s - loss: 2.4294 - acc: 0.2941
Epoch 21/500
 - 0s - loss: 2.4251 - acc: 0.2941
E

Epoch 171/500
 - 0s - loss: 0.8259 - acc: 0.6471
Epoch 172/500
 - 0s - loss: 0.8198 - acc: 0.6471
Epoch 173/500
 - 0s - loss: 0.8140 - acc: 0.6471
Epoch 174/500
 - 0s - loss: 0.8083 - acc: 0.6471
Epoch 175/500
 - 0s - loss: 0.8028 - acc: 0.6471
Epoch 176/500
 - 0s - loss: 0.7975 - acc: 0.6471
Epoch 177/500
 - 0s - loss: 0.7924 - acc: 0.6471
Epoch 178/500
 - 0s - loss: 0.7875 - acc: 0.6471
Epoch 179/500
 - 0s - loss: 0.7828 - acc: 0.6471
Epoch 180/500
 - 0s - loss: 0.7782 - acc: 0.6471
Epoch 181/500
 - 0s - loss: 0.7738 - acc: 0.6471
Epoch 182/500
 - 0s - loss: 0.7696 - acc: 0.6471
Epoch 183/500
 - 0s - loss: 0.7655 - acc: 0.6471
Epoch 184/500
 - 0s - loss: 0.7616 - acc: 0.6471
Epoch 185/500
 - 0s - loss: 0.7578 - acc: 0.6471
Epoch 186/500
 - 0s - loss: 0.7541 - acc: 0.6471
Epoch 187/500
 - 0s - loss: 0.7506 - acc: 0.6471
Epoch 188/500
 - 0s - loss: 0.7472 - acc: 0.6471
Epoch 189/500
 - 0s - loss: 0.7440 - acc: 0.6471
Epoch 190/500
 - 0s - loss: 0.7408 - acc: 0.6471
Epoch 191/500
 - 0s 

 - 0s - loss: 0.6460 - acc: 0.6471
Epoch 339/500
 - 0s - loss: 0.6459 - acc: 0.6471
Epoch 340/500
 - 0s - loss: 0.6458 - acc: 0.6471
Epoch 341/500
 - 0s - loss: 0.6458 - acc: 0.6471
Epoch 342/500
 - 0s - loss: 0.6457 - acc: 0.6471
Epoch 343/500
 - 0s - loss: 0.6456 - acc: 0.6471
Epoch 344/500
 - 0s - loss: 0.6455 - acc: 0.6471
Epoch 345/500
 - 0s - loss: 0.6454 - acc: 0.6471
Epoch 346/500
 - 0s - loss: 0.6454 - acc: 0.6471
Epoch 347/500
 - 0s - loss: 0.6453 - acc: 0.6471
Epoch 348/500
 - 0s - loss: 0.6452 - acc: 0.6471
Epoch 349/500
 - 0s - loss: 0.6451 - acc: 0.6471
Epoch 350/500
 - 0s - loss: 0.6451 - acc: 0.6471
Epoch 351/500
 - 0s - loss: 0.6450 - acc: 0.6471
Epoch 352/500
 - 0s - loss: 0.6449 - acc: 0.6471
Epoch 353/500
 - 0s - loss: 0.6449 - acc: 0.6471
Epoch 354/500
 - 0s - loss: 0.6448 - acc: 0.6471
Epoch 355/500
 - 0s - loss: 0.6447 - acc: 0.6471
Epoch 356/500
 - 0s - loss: 0.6447 - acc: 0.6471
Epoch 357/500
 - 0s - loss: 0.6446 - acc: 0.6471
Epoch 358/500
 - 0s - loss: 0.6445

In [48]:
# generate a sequence from a language model
def generate_seq1(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words    
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')[:, 0]
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

In [74]:
predic = (generate_seq1(model2, tokenizer1, max_length-1, "planlamasina r",1))
print('Next Word Prediction : %s' % predic)


Next Word Prediction : planlamasina r a


In [78]:
testset1= "planlamasina r"
model0 = unigram(Xx)
perplexity(testset1, model0)

100.0